In [46]:
!pip install qdrant_client -q
!pip install sentence_transformers -q
!pip install fsspec -q
!pip install openai==1.11.1 -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [23]:
import pandas as pd
data = pd.read_csv('netflix_titles.csv').sample(700).to_dict('records')
len(data)

700

In [10]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [11]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/Users/upw_jmengel/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [24]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="netflix",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/ky/rwwshgbj7393p7w4fv4cjgsr0000gn/T/ipykernel_88407/504723424.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [29]:
# vectorize!
qdrant.upload_points(
    collection_name="netflix",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc['description']).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [31]:
hits = qdrant.search(
    collection_name="netflix",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'show_id': 's5555', 'type': 'TV Show', 'title': 'Five Came Back', 'director': 'Laurent Bouzereau', 'cast': 'Steven Spielberg, Guillermo del Toro, Lawrence Kasdan, Francis Ford Coppola, Paul Greengrass, Meryl Streep', 'country': nan, 'date_added': 'March 31, 2017', 'release_year': 2017, 'rating': 'TV-MA', 'duration': '1 Season', 'listed_in': 'Docuseries', 'description': 'Five acclaimed contemporary directors tell the story of five legendary Hollywood filmmakers who enlisted in the armed forces to document World War II.'} score: 0.4738601517542096
{'show_id': 's8217', 'type': 'TV Show', 'title': 'The Bomb Squad', 'director': nan, 'cast': 'Christopher Eccleston', 'country': 'United Kingdom', 'date_added': 'February 1, 2019', 'release_year': 2011, 'rating': 'TV-MA', 'duration': '1 Season', 'listed_in': 'British TV Shows, Docuseries', 'description': 'This documentary follows British bomb disposal teams in Afghanistan and the dangers they face in their methodical lives on the front lines of

In [32]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [63]:
from openai import OpenAI
import getpass
client = OpenAI(api_key=getpass.getpass())

def get_response(prompt):
  response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        n=1,
        stop=None,
        temperature=0.3,
        messages=[
            {"role": "system", "content": "You are chatbot, a tv and movie specialist. Your top priority is to help guide users into selecting amazing tv shows and movies and guide them with their requests."},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": str(search_results)}
        ]
  )
  return response.choices[0].message.content

········


In [65]:
get_response("Suggest me an amazing world war 2 movie (not tv show) that will teach me something about the war.")

'I recommend the movie "Schindler\'s List" directed by Steven Spielberg. This powerful film tells the true story of Oskar Schindler, a German businessman who saved over a thousand Polish-Jewish refugees during the Holocaust by employing them in his factories. It provides a moving and educational portrayal of the atrocities of World War II and the resilience of the human spirit.'